# PATEGAN example

In this example we will be comparing the YData-Synthetic PATE-GAN implementation to the one from the original authors in the [mlforhealthlabpub](https://github.com/vanderschaarlab/mlforhealthlabpub/tree/main/alg/pategan) package. Since this package has a lot of dependencies and uses TensorFlow 1, we recommend that you create a new environment and follow their setup instructions available [here](https://github.com/vanderschaarlab/mlforhealthlabpub/blob/main/doc/install.md).

## Introduction
To run this comparison we have executed `mlforhealthlabpub`'s implementation via the main script, together with their fake dataset script used for random dataset generation. With this utility script we have produced a train dataset used to train both synthesizers. Both synthesizers are defined according to the same set of parameters. After producing two versions of synthetic datasets we will use [Pandas Profiling](https://github.com/ydataai/pandas-profiling) to compare the outputs regarding fidelity.

### Import the required packages

In [1]:
import pandas as pd
import pandas_profiling as pp

from ydata_synthetic.synthesizers.regular import PATEGAN

2022-05-16 18:19:02.584078: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


### Load the train dataset and the synthetic dataset from the original dataset

In [2]:
dir = '../../data/'
train_data = pd.read_csv(dir+'train_dataset.csv', index_col=0)
orig_synth = pd.read_csv(dir+'orig_synth.csv', index_col=0)

### Train the YData-Synthetic synthesizer

In [3]:
from ydata_synthetic.synthesizers import ModelParameters

num_cols = train_data.columns.to_list()
cat_cols = []

#Defining the training parameters
noise_dim = 128
dim = 4*len(train_data.columns)
batch_size = 64

log_step = 100
learning_rate = [5e-4, 3e-3]
beta_1 = 0.5
beta_2 = 0.9
models_dir = './cache'

gan_args = ModelParameters(batch_size=batch_size,
                           lr=learning_rate,
                           betas=(beta_1, beta_2),
                           noise_dim=noise_dim,
                           layers_dim=dim)

#  PATEGAN specific arguments
n_moments = 20
n_teacher_iters = 1
n_student_iters = 1
n_teachers = 10
##  Privacy/utility tradeoff specification
target_delta = 1e-5
target_epsilon = 1
lap_scale = 1e-2

model = PATEGAN

synthesizer = model(gan_args, n_teachers, target_delta, target_epsilon)
synthesizer.train(train_data, num_cols, cat_cols,
                  n_teacher_iters, n_student_iters, n_moments, lap_scale)

synthesizer.save('pate_test.pkl')

synthesizer = model.load('pate_test.pkl')
ydata_synth = synthesizer.sample(train_data.shape[0])


2022-05-16 18:19:03.613787: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-05-16 18:19:03.614447: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-05-16 18:19:03.643719: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2070 Super computeCapability: 7.5
coreClock: 1.38GHz coreCount: 40 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 417.29GiB/s
2022-05-16 18:19:03.643755: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-05-16 18:19:03.646305: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2022-05-16 18:19:03.646372: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2022-05-16 18:19:03.666574: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-16 18:19:03.667386: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-05-16 18:19:03.667959: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2070 Super computeCapability: 7.5
coreClock: 1.38GHz coreCount: 40 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 417.29GiB/s
2022-05-16 18:19:03.668011: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-05-16 18:19:03.668032: I tensorflow/stream_executor/platform/default/dso_load

Step : 0 Loss SD : 6.87e-01 Loss G : 7.65e-01 Epsilon : 8.44e-01
Step : 1 Loss SD : 7.01e-01 Loss G : 7.10e-01 Epsilon : 1.11e+00


Synthetic data generation: 100%|██████████| 157/157 [00:00<00:00, 734.24it/s]


### Profiling the synthetic samples

In [7]:
train_prof = pp.ProfileReport(train_data, title='Train data')

orig_prof = pp.ProfileReport(orig_synth, title='Original PATEGAN implementation synthetic samples')
ydata_prof = pp.ProfileReport(ydata_synth, title='YData-Synthetic PATEGAN implementation synthetic samples')

In [8]:
train_prof.to_widgets()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
orig_prof.to_widgets()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
ydata_prof.to_widgets()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]